# Controle da carteira de ações (Swing Trade)

### Referências para o programa.

[Link](https://gspread.readthedocs.io/en/latest/api.html#module-gspread.utils) de uma documentação
das planilhas do google.

[Link](https://www.twilio.com/blog/2017/02/an-easy-way-to-read-and-write-to-a-google-spreadsheet-in-python.html) descrevendo como fazer para pegar as autenticações do google spreadsheet.

In [ ]:
ano = '2020'

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import locale
locale.setlocale(locale.LC_ALL, 'pt_BR.utf8')

In [ ]:
sns.set(style="white")

plt.rcParams['axes.titlesize'] = 20
plt.rcParams['axes.labelsize'] = 20
plt.rcParams['axes.linewidth'] = 3
plt.rcParams['axes.labelpad'] = 15

plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['xtick.major.size'] = 8
plt.rcParams['xtick.major.width'] = 3

plt.rcParams['ytick.labelsize'] = 16
plt.rcParams['ytick.major.size'] = 8
plt.rcParams['ytick.major.width'] = 3

plt.rcParams['legend.fontsize'] = 14
plt.rcParams['legend.markerscale'] = 1

plt.rcParams['lines.markersize'] = 9
plt.rcParams['lines.linewidth'] = 2

### Credênciais para a API do Google

In [ ]:
scope = ['https://www.googleapis.com/auth/spreadsheets',
        'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('cred.json', scope)
client = gspread.authorize(creds)

### Lê os datasets

In [ ]:
df_ativos = pd.read_csv('Dataset_ações_' + ano + '.csv')
df_ativos['Data'] = pd.to_datetime(df_ativos['Data'], format="%Y/%m/%d")
df_ativos.head()

In [ ]:
df_proventos = pd.read_excel('Proventos - ' + ano + '.xls')
df_proventos.head()

# Funções

In [ ]:
def carteira_de_acoes_atual(df, offline=False, last_date=None):
    
    if last_date != None:
        last_date = pd.to_datetime(last_date, infer_datetime_format=True, dayfirst=True)
        df = df[df['Data'] <= last_date]
    
    carteira = pd.DataFrame(columns=df.columns)

    cods = df['Código'].unique()
    
    for cod in cods:

        last_idx = df[df[['Código']] == cod].last_valid_index() 

        carteira = carteira.append(df.iloc[last_idx], ignore_index=True)

    carteira = carteira[carteira['Quantidade Atual'] != 0.0]
    
    carteira.sort_values(by='Data', ascending=True, inplace=True)
    
    drop_columns = ['Especificação', 'Tipo de Execução', 'Quantidade de C/V', 'Mercado', 'Preço de C/V']
    
    carteira.drop(drop_columns, inplace=True, axis=1)
    
    carteira.rename(columns={'Preço Atual': '<Preço>'}, inplace=True)
    
    carteira.reset_index(drop=True, inplace=True)
    
    carteira['<Valor Total>'] = carteira['Quantidade Atual'] * carteira['<Preço>']
    
    if offline == False:
    
        carteira['Preço Atual'] = get_market_values(carteira['Código'])

        carteira['Valor Total Atual'] = carteira['Quantidade Atual'] * carteira['Preço Atual']

        carteira['L/P'] = carteira['Quantidade Atual'] * ( carteira['Preço Atual'] - carteira['<Preço>'] )

        carteira['L/P (%)'] = (carteira['L/P'] / (carteira['<Preço>'] * carteira['Quantidade Atual'])) * 100.0

        carteira['L/P'] = carteira['L/P'].round(2)

        carteira['L/P (%)'] = carteira['L/P (%)'].round(2) 

        carteira['<Preço>'] = carteira['<Preço>'].round(2)


        cols_ordenadas = ['Data', 'Código', 'Quantidade Atual', '<Preço>', '<Valor Total>',
                          'Preço Atual', 'Valor Total Atual', 'L/P', 'L/P (%)']

    elif offline == True:
        
        cols_ordenadas = ['Data', 'Código', 'Quantidade Atual', '<Preço>', '<Valor Total>']
        
    return carteira[cols_ordenadas]

In [ ]:
def get_market_values(names):
    
    market_values = client.open('Google_Finance').sheet1

    size = names.shape[0]
    
    cell_list = market_values.range('A1:A' + str(size))

    for i, cell in enumerate(cell_list):
        cell.value = names[i]
        market_values.update_acell('B' + str(i + 1), '=GOOGLEFINANCE(A' + str(i + 1) + ';"price")')

    market_values.update_cells(cell_list);

    market_values = market_values.col_values(2)[0:size]
    
    market_values = [float(market_values[i].replace(',', '.')) for i in range(size)]
      
    return np.asarray(market_values)

In [ ]:
def update_carteira_online(carteira):

    carteira['Data'] = carteira['Data'].dt.strftime('%d-%m-%Y')

    carteira['Preço Atual'] = ['=GOOGLEFINANCE(B' + str(i) + '; "price")' for i in range(2, carteira.shape[0]+2)]

    carteira['<Valor Total>'] = ['=D' + str(i) + ' * C' + str(i) for i in range(2, carteira.shape[0]+2)]

    carteira['Valor Total Atual'] = ['=F' + str(i) + ' * C' + str(i) for i in range(2, carteira.shape[0]+2)]

    carteira['L/P'] = ['=G' + str(i) + ' - E' + str(i) for i in range(2, carteira.shape[0]+2)]

    carteira['L/P (%)'] = ['=(H' + str(i) + ' / E' + str(i) + ')' for i in range(2, carteira.shape[0]+2)]

    google_sheet = client.open('Carteira de Ações').sheet1
    google_sheet.clear()

    google_sheet.insert_row(carteira.columns.to_list(), index=1)

    for row in range(carteira.shape[0]):
        google_sheet.insert_row(carteira.iloc[row].to_list(), index=row+2, value_input_option='USER_ENTERED')

### Carteira Atual

In [ ]:
sheet = client.open("Carteira de Ações").sheet1

In [ ]:
df_ativos

In [ ]:
carteira = carteira_de_acoes_atual(df_ativos, last_date = '02/02/2020')
carteira

In [ ]:
print('Lucro ao zerar a carteira: {0}'.format(carteira['L/P'].sum()))

In [ ]:
print('Valor Total Investido: {0}'.format(carteira['Valor Total Atual'].sum().round(2)))

### Atualiza a carteira online

In [ ]:
#update_carteira_online(carteira.copy())

# Visualizações

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

## Carteira Atual

In [ ]:
def f(col_name):
    
    plt.figure(figsize=[15, 9])
    
    df = carteira
    
    if col_name == 'L/P':
        
        tot = pd.DataFrame(data={'Código': ['Total'], col_name: [carteira[col_name].sum().round(2)]})

        df = carteira.append(tot, ignore_index=True)

    
        
    palette_c = ['r' if c < 0.0 else 'b' for c in df[col_name]]

    g = sns.barplot(data = df, x = 'Código', y = col_name, palette=palette_c)

    for index, row in df[[col_name]].reset_index().values:
        g.text(index, row, np.round(row, 2), color='black', ha="center", va='bottom',
               fontsize='large', fontweight='bold')


    plt.hlines(y=0, xmax=df.shape[0], xmin=-1)
    plt.title('Carteira Atual')
    plt.ylabel(col_name)
    plt.xlabel('')
    plt.xticks(rotation=35)
    plt.show()

interactive(f, col_name=carteira.columns[2:].to_list())

## Lucro por mês

In [ ]:
lucro_df = df_ativos.loc[(df_ativos['L/P'] != 0.0), ['Data', 'Código', 'L/P']]
lucro_df['Mês'] = lucro_df['Data'].dt.strftime('%b')
lucro_df = lucro_df.groupby(['Mês', 'Código'], sort=False).sum()

In [ ]:
months = lucro_df.reset_index()['Mês'].unique()

In [ ]:
def f(month):
    
    df = lucro_df.loc[month].reset_index()
    
    tot = pd.DataFrame(data={'Código': ['Total'], 'L/P': [df['L/P'].sum().round(2)]})
    
    df = df.append(tot, ignore_index=True)
    
    palette_c = ['r' if c < 0.0 else 'b' for c in df['L/P']]
    
    plt.figure(figsize=[15, 9])

    g = sns.barplot(data = df, x = 'Código', y = 'L/P', palette=palette_c)

    for index, row in df[['L/P']].reset_index().values:
        g.text(index, row, np.round(row, 2), color='black', ha="center", va='bottom',
           fontsize='large', fontweight='bold')
        
    plt.hlines(y=0, xmax=df.shape[0], xmin=-1)
    plt.ylabel('Lucro  (R$)')
    plt.xlabel('')
    plt.xticks(rotation=35)
    plt.title('Lucro mensal')
    
    plt.show()

interactive(f, month=months)

## Lucro durante o ano

In [ ]:
lucro_mes = df_ativos.loc[(df_ativos['L/P'] != 0.0), ['Data', 'L/P']]
lucro_mes['Mês'] = lucro_mes['Data'].dt.strftime('%b')
lucro_mes = lucro_mes.groupby(['Mês'], sort=False).sum().reset_index()

In [ ]:
plt.figure(figsize=[15, 9])

tot = pd.DataFrame(data={'Mês': ['Ano'], 'L/P': [lucro_mes['L/P'].sum().round(2)]})
    
df = lucro_mes.append(tot, ignore_index=True)

palette_c = ['r' if c < 0.0 else 'b' for c in df['L/P']]

g = sns.barplot(data = df, x = 'Mês', y = 'L/P', palette=palette_c)

for index, row in df[['L/P']].reset_index().values:
    g.text(index, row, np.round(row, 2), color='black', ha="center", va='bottom',
           fontsize='large', fontweight='bold')
    
plt.hlines(y=0, xmax=df.shape[0], xmin=-1)
plt.ylabel('Lucro  (R$)')
plt.xlabel('')
plt.title('Lucro no ano')
plt.xticks(rotation=35)
plt.show()
del df

## Proventos

In [ ]:
prov = df_proventos.groupby(['Código']).sum().reset_index()

In [ ]:
plt.figure(figsize=[15, 9])

tot = pd.DataFrame(data={'Código': ['Ano'], 'Valor': [prov['Valor'].sum().round(2)]})
    
df = prov.append(tot, ignore_index=True)

g = sns.barplot(data = df, x = 'Código', y = 'Valor', color='b')

for index, row in df[['Valor']].reset_index().values:
    g.text(index, row, np.round(row, 2), color='black', ha="center", va='bottom',
           fontsize='large', fontweight='bold')

plt.hlines(y=0, xmax=df.shape[0], xmin=-1)
plt.title('Proventos')
plt.ylabel('Lucro  (R$)')
plt.xlabel('')
plt.show()